# Scraping Muthead player Data
#### Matthew Johnson, March 24, 2018


In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import requests
import warnings; warnings.simplefilter('ignore')  
import time

In [6]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) \
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36'}

link_start = 'https://www.muthead.com/18/players/' # LINK PREFIX/POSTFIX
link_end = '/full-ratings/'

chem_list = ['Read and React', 'Physical Front', 'Toughness', 'Blanket Coverage', \
             'Go Deep', 'Move the Sticks', 'Pound the Rock', 'Secure Tackler', \
             'Playmaker', 'Pick Specialist']
attributes = ['SPD', 'STR', 'AGI', 'ACC', 'AWR', 'CTH', \
              'JMP', 'STA', 'INJ', 'TAK', 'POW', 'PWM', 'FNM', \
              'BKS', 'PUR', 'PRC', 'MCV', 'ZCV', 'PRS', 'KPW', 'KAC', 'KR']


def retrieve_stats2(player_id):
    mut_url = link_start + str(player_id) + link_end
    
    r = requests.get(mut_url, headers=headers)

    soup = BeautifulSoup(r.text, 'lxml')
    
    print(soup.title.text)
    
    st = soup.get_text()
    doc = st.split('\n')

    curr_chems = []
    attributes = []
    ratings = []
     
    df = pd.DataFrame(columns=attributes, index=[0])
    df['player'] = '-'
    df['id']=0
    df['ovr']=0
    
    df['pos']='-'
    df['size']='-'
    df['chem_1']='-'
    df['chem_2']='-'
    df['chem_3']='-'

    df['ovr'][0] = doc[668] # ovr    
    df['pos'][0] = doc[670] # position
    df['size'][0] = doc[710] # size
    
    for w in range(0,len(chem_list)):
        df [ chem_list[w] ] = 0
    
    for j in range(749,760):
        if doc[j] in chem_list:
            x = chem_list[chem_list.index(doc[j])]
            curr_chems.append ( x )
            df[x][0]=1

    for i in range(770,835):
        s = doc[i].split('  ')
        print('s, i', s, ' ' , i)
        
        if ((len(s)==2) & (len(s[0])==2)):
            
            if s[1] not in attributes:
                attributes.append(s[1])
                df[ s[1] ]=0
            ratings.append(s[0])
            attr = s[1]
            df[attr][0] = s[0]

    df['player'] = soup.title.text[15:-31]
    df['id'] = player_id
    
    return df

In [278]:
saved_columns = df.columns
chem_list = ['Read and React', 'Physical Front', 'Toughness', 'Blanket Coverage', \
             'Go Deep', 'Move the Sticks', 'Pound the Rock', 'Secure Tackler', \
             'Playmaker', 'Pick Specialist']

columbos = ['player', 'id', 'ovr', 'pos', 'size', 'SPD', 'STR', 'AGI', 'ACC', 'AWR', 'CTH', 'JMP', 'STA', 'INJ', 'TAK',
       'POW', 'PWM', 'FNM', 'BKS', 'PUR', 'PRC', 'MCV', 'ZCV', 'PRS', 'KPW',
       'KAC', 'KR',  'TRK', 'ELU', 'BCV', 'SFA', 'SPM', 'JKM', 'CAR', 'RTE', 'CIT',
       'SPC', 'RLS', 'THP', 'TAS', 'TAM', 'TAD', 'TOR', 'PAC', 'RBK', 'PBK',
       'IMP', 'Read and React', 'Physical Front', 'Toughness', 'Blanket Coverage', \
             'Go Deep', 'Move the Sticks', 'Pound the Rock', 'Secure Tackler', \
             'Playmaker', 'Pick Specialist']

In [ ]:
pids = [24211, 61, 32207, 32165] # player IDs

start = time.time() # INITIALIZE TIMER FOR COMPUTING TIME
print("timestart")

player_data = pd.DataFrame(columns=columbos)

for i in range(0,len(pids)):
    temp = retrieve_stats2( pids[i])
    player_data = pd.concat([player_data, temp], axis=0, ignore_index=True)
    #time.sleep(2)
    
end = time.time()
print('Time in seconds for completion:\t', end-start)

In [289]:
player_data

,ACC,AGI,AWR,BCV,BKS,Blanket Coverage,CAR,CIT,CTH,ELU,...,Toughness,ZCV,chem_1,chem_2,chem_3,id,ovr,player,pos,size
0,90,89,90,80,78,0,78,79,82,82,...,0,91,-,-,-,24211,92,Earl Thomas III,FS,
1,98,96,95,83,86,0,83,84,85,88,...,1,97,-,-,-,61,99,Night Train Lane,CB,Detroit Lions
2,98,94,94,90,15,0,75,95,96,93,...,0,45,-,-,-,32207,98,Andre Reed,WR,"Ht: 6' 2"" Wt: 190"
3,96,94,95,95,40,0,70,92,97,94,...,1,27,-,-,-,32165,98,Randy Moss,WR,"Ht: 6' 4"" Wt: 210"


In [3]:
link_start = 'https://www.muthead.com/18/players/' # LINK PREFIX/POSTFIX
link_end = '/full-ratings/'

chem_list = ['Read and React', 'Physical Front', 'Toughness', 'Blanket Coverage', \
             'Go Deep', 'Move the Sticks', 'Pound the Rock', 'Secure Tackler', \
             'Playmaker', 'Pick Specialist']
attributes = ['SPD', 'STR', 'AGI', 'ACC', 'AWR', 'CTH', \
              'JMP', 'STA', 'INJ', 'TAK', 'POW', 'PWM', 'FNM', \
              'BKS', 'PUR', 'PRC', 'MCV', 'ZCV', 'PRS', 'KPW', 'KAC', 'KR']

In [11]:
r = requests.get(mut_url1, headers=headers)

soup = BeautifulSoup(r.text, 'lxml')
    
print(soup.title.text)
    
st = soup.get_text()
doc = st.split('\n')

Full Ratings - Andre Reed - Madden 18 Players - Muthead 


In [13]:
for j in range(290,len(doc)):
    a = doc[j]
    if (len(a) > 0):
        print('j: ', j, ', doc[j]: ', a)

j:  290 , doc[j]:                  
j:  295 , doc[j]:  
j:  296 , doc[j]:                     M15 Sets
j:  297 , doc[j]:      
j:  298 , doc[j]:                  
j:  305 , doc[j]:  
j:  306 , doc[j]:                     Madden 25
j:  307 , doc[j]:      
j:  308 , doc[j]:                  
j:  313 , doc[j]:  
j:  314 , doc[j]:                     M25 Players
j:  315 , doc[j]:      
j:  316 , doc[j]:                  
j:  321 , doc[j]:  
j:  322 , doc[j]:                     M25 Items
j:  323 , doc[j]:      
j:  324 , doc[j]:                  
j:  329 , doc[j]:  
j:  330 , doc[j]:                     M25 Collections
j:  331 , doc[j]:      
j:  332 , doc[j]:                  
j:  341 , doc[j]:  
j:  342 , doc[j]:                     Prices
j:  343 , doc[j]:      
j:  344 , doc[j]:                  
j:  349 , doc[j]:  
j:  350 , doc[j]:                     Gauntlet
j:  351 , doc[j]:      
j:  352 , doc[j]:                  
j:  357 , doc[j]:  
j:  358 , doc[j]:                     Leagues

In [19]:
tempatts = []
index_of_traits = 1121

for c in range (index_of_traits, index_of_traits+15):

    s = doc[c].split('  ')
    print('s, i', s, ' ' , c)

    if s[0] is 'Clutch':
        print('clutch: ', s[1] )
    if s[0] is 'Fights':
        print('fight for extra: ', s[1:])

s, i ['Player Traits']   1121
s, i ['']   1122
s, i ['Clutch Yes']   1123
s, i ['Covers Ball Brace vs. Big Hitters']   1124
s, i ['Fights for Extra Yards Yes']   1125
s, i ['Makes Aggressive Catches No']   1126
s, i ['Makes RAC Catches Yes']   1127
s, i ['Makes Possession Catches Yes']   1128
s, i ['Drops Open Passes No']   1129
s, i ['Makes Sideline Catches Yes']   1130
s, i ['High Motor Yes']   1131
s, i ['Penalty Normal']   1132
s, i ['']   1133
s, i ['']   1134
s, i ['']   1135


In [24]:
souptex = soup.text
if souptex.find('Clutch Yes')>0:
    dat['clutch'] = 1
if souptex.find('Fights for Extra Yards Yes')>0:
    dat['fights_for_extra'] = 1  

15858

In [27]:
souptex.find('Auction')

-1